In [1]:
import shutil
from glob import glob
import pandas as pd
import os
import numpy as np

In [2]:
meta_info_df = pd.read_pickle('../../../data/tumor_img_meta_info.pkl')
train_test_split_df = pd.read_pickle('../../../data/train_val_test_split.pkl')

In [3]:
meta_info_df = meta_info_df.merge(train_test_split_df, on='img_id')

In [4]:
meta_info_df.head()

,img_id,level_5_img_size,level_5_mask_area,level_dimensions,level_downsamples,mask_img_filename,mask_proportion,slide_img_filename,type
0,001,21123072,11116,"((97792, 221184), (48896, 110592), (24448, 552...","(1.0, 2.0, 4.0, 8.0, 16.0, 32.0, 64.0, 128.0, ...",tumor_001_mask.tif,0.052625,tumor_001.tif,val
1,002,20976384,938,"((97792, 219648), (48896, 109824), (24448, 549...","(1.0, 2.0, 4.0, 8.0, 16.0, 32.0, 64.0, 128.0, ...",tumor_002_mask.tif,0.004472,tumor_002.tif,test
2,005,20976384,2323,"((97792, 219648), (48896, 109824), (24448, 549...","(1.0, 2.0, 4.0, 8.0, 16.0, 32.0, 64.0, 128.0, ...",tumor_005_mask.tif,0.011074,tumor_005.tif,train
3,012,20585216,1115,"((97792, 215552), (48896, 107776), (24448, 538...","(1.0, 2.0, 4.0, 8.0, 16.0, 32.0, 64.0, 128.0, ...",tumor_012_mask.tif,0.005417,tumor_012.tif,val
4,016,21123072,156981,"((97792, 221184), (48896, 110592), (24448, 552...","(1.0, 2.0, 4.0, 8.0, 16.0, 32.0, 64.0, 128.0, ...",tumor_016_mask.tif,0.743173,tumor_016.tif,train


In [5]:
SOURCE_PARTITION_DIR = '/home/sjb/Projects/Columbia/Applied_DL/CamelyonProject/data/temp_partitions_dir/'
DEST_DIR = '/home/sjb/Projects/Columbia/Applied_DL/CamelyonProject/data/zoom_5_60_60_partition'

In [6]:
# for split_type in ['train', 'val', 'test']:
    
#     dest_sub_dir = os.path.join(DEST_DIR, split_type)
    
#     if not os.path.exists(dest_sub_dir):
#         os.mkdir(dest_sub_dir)
#         print('Creating directory', dest_sub_dir)
    
#     filtered_df = meta_info_df[meta_info_df['type'] == split_type]
#     img_id_list = filtered_df['img_id'].values
    
#     for img_type in ['slide', 'mask']:
#         dest_sub_sub_dir = os.path.join(dest_sub_dir, img_type)
        
#         if not os.path.exists(dest_sub_sub_dir):
#             os.mkdir(dest_sub_sub_dir)
#             print('Creating directory', dest_sub_sub_dir)
        
#         for img_id in img_id_list:
#             file_prefix = 'tumor_{}_{}_split'.format(img_type, img_id)
            
#             if img_type == 'slide':
#                 source_file_paths = glob(os.path.join(SOURCE_PARTITION_DIR, file_prefix) + '*.png')
#             else:
#                 source_file_paths = glob(os.path.join(SOURCE_PARTITION_DIR, file_prefix) + '*.npy')
            
#             for source_file_path in source_file_paths:
#                 basename = os.path.basename(source_file_path)
#                 dest_file_path = os.path.join(dest_sub_sub_dir, basename)
#                 shutil.copy2(source_file_path, dest_file_path)

## Sanity Check

In [7]:
infos = []

In [8]:
for split_type in ['train', 'val', 'test']:
    sub_dir = os.path.join(DEST_DIR, split_type)
    
    for img_type in ['slide', 'mask']:
        sub_sub_dir = os.path.join(sub_dir, img_type)
        
        png_files = glob(sub_sub_dir + '/*.png')
        npy_files = glob(sub_sub_dir + '/*.npy')
        
        if img_type == 'slide':
            valid_files = png_files
        else:
            valid_files = npy_files
            
        uniq_imgs = np.unique([os.path.basename(x).split('_')[2] for x in valid_files])
        
        info = {
            'split_type': split_type,
            'img_type': img_type,
            'n_png': len(png_files),
            'n_npy': len(npy_files),
            'uniq_imgs': uniq_imgs
        }
        infos.append(info)
        
        

In [9]:
info_df = pd.DataFrame(infos)

In [10]:
info_df

,img_type,n_npy,n_png,split_type,uniq_imgs
0,slide,0,50339,train,"[005, 016, 019, 023, 035, 059, 075, 081, 084, ..."
1,mask,50339,0,train,"[005, 016, 019, 023, 035, 059, 075, 081, 084, ..."
2,slide,0,22800,val,"[001, 012, 031, 064]"
3,mask,22800,0,val,"[001, 012, 031, 064]"
4,slide,0,15089,test,"[002, 057, 078, 091]"
5,mask,15089,0,test,"[002, 057, 078, 091]"


In [11]:
all_files = glob(SOURCE_PARTITION_DIR + '*')

In [12]:
len(all_files)

176456

In [15]:
info_df[['n_npy', 'n_png']].sum().sum()

176456

In [23]:
info_df.iloc[4]['uniq_imgs']

array(['002', '057', '078', '091'], dtype='<U3')

In [22]:
meta_info_df[meta_info_df['type'] == 'test']['img_id'].values

array(['002', '057', '078', '091'], dtype=object)